## Basic chatbot with document analysis

A basic python notebook that:
- Loads the required environment and libraries
- Checks for HuggingFace token key in the environment
- Uses the token key to call the model hosted on HuggingFace
- Congfigures document reader
- Defines the chat function
    - Sets the system prompt
    - Includes the chat history for consersation
    - streams the response for visual experience
- Creates an interface using Gradio

In [3]:
# Install required packages if not already installed
%pip install docling nest-asyncio


Note: you may need to restart the kernel to use updated packages.


c:\Users\joshu\projects\llm_engineering\.venv\Scripts\python.exe: No module named pip


In [5]:
# Import libraries

import os
import shutil
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI
from docling.document_converter import DocumentConverter
import gradio as gr
import nest_asyncio

# Enable nested event loops for Jupyter
nest_asyncio.apply()

ModuleNotFoundError: No module named 'docling'

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)

# openai_api_key = os.getenv('OPENAI_API_KEY')

#if openai_api_key:
#    print(f"OpenAI API Key exists and begins with {openai_api_key[:8]}")
#else:
#    print("OpenAI API Key not set")

HF_TOKEN = os.getenv("HF_TOKEN")

if HF_TOKEN is None:
    print("No HF_TOKEN set.")
    raise ValueError("Please set HF_TOKEN in your .env file")
else:
    print(f"HF_TOKEN exists and begins with {HF_TOKEN[:8]}")

HF_TOKEN exists and begins with hf_JoUld


In [ ]:
# Setup the OpenAI compatible client pointing at Hugging Face Router
# This uses HuggingFace's infrastructure, not OpenAI's models.

client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_TOKEN,
)

# Setup model hosted on HuggingFace

MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"

In [4]:
# Setup converter for documents

converter = DocumentConverter()

# Folder to keep uploaded documents (so you can commit them if you want)

UPLOAD_DIR = Path("uploaded_docs")
UPLOAD_DIR.mkdir(exist_ok=True)

# Define function for text extraction from the file

def extract_text_from_file(uploaded_file):
    """
    Take a Gradio UploadedFile, save it into UPLOAD_DIR,
    run Docling on it, return markdown text.
    """
    if uploaded_file is None:
        return ""

    # Save a copy of the uploaded file into our repo folder
    dest_path = UPLOAD_DIR / uploaded_file.name
    shutil.copy(uploaded_file.name, dest_path)

    # Use Docling to convert to a rich document, then export as markdown
    result = converter.convert(str(dest_path))
    doc_text = result.document.export_to_markdown()

    # You could also use export_to_text() if you prefer plain text
    return doc_text


In [ ]:
# Define chat function to interact with model

def chat(message, history, uploaded_file):
    """
    Gradio ChatInterface callback.

    Parameters
    ----------
    message : str
        The latest user message.
    history : list[dict]
        Previous messages in OpenAI format:
        [{"role": "user"/"assistant", "content": "..."}, ...]
        (because we use type="messages" in ChatInterface).
    uploaded_file : str | None
        Path to the uploaded file (because we use type="filepath" in gr.File),
        or None if no file is provided this turn.
    """

    system_message = (
        "You are a helpful assistant. If a document is provided, "
        "use it as the main context for your answers."
    )

    # Start with system message + previous conversation
    messages = [{"role": "system", "content": system_message}] + (history or [])

    # Build the current user message, optionally including file contents
    if uploaded_file is not None:
        try:
            # Use Docling to extract text from the document
            from pathlib import Path
            file_path = Path(uploaded_file)
            result = converter.convert(str(file_path))
            doc_text = result.document.export_to_markdown()
        except Exception as e:
            doc_text = f"[Error processing file {uploaded_file}: {e}]"

        user_content = (
            "Here is the document content:\n\n"
            f"{doc_text}\n\n"
            f"Now answer my question about this document:\n{message}"
        )
    else:
        user_content = message

    messages.append({"role": "user", "content": user_content})

    # Call the model with streaming
    stream = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        stream=True,
        max_tokens=512,
    )

    response = ""
    for chunk in stream:
        # Be robust to empty/partial chunks
        if not chunk.choices:
            continue
        delta = chunk.choices[0].delta
        if not delta or delta.content is None:
            continue

        response += delta.content
        yield response


In [ ]:
# Create a chat interface using Gradio,
# interface launches in new browser window

view = gr.ChatInterface(
    fn=chat,
    type="messages",
    title="LLaMA + Document Chat",
    description="Chat with LLaMA. Optionally upload a document and ask about it.",
    additional_inputs=[
        gr.File(label="Upload a document (optional)", type="filepath")
    ],
)

# Launch the interface
# Using share=False and inbrowser=True for Jupyter compatibility

view.launch(inbrowser=True, share=False)

2025-11-17 01:09:06,477 - INFO - HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-11-17 01:09:06,501 - INFO - HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


2025-11-17 01:09:06,991 - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2025-11-17 01:09:24,280 - INFO - detected formats: [<InputFormat.CSV: 'csv'>]
2025-11-17 01:09:24,302 - INFO - Going to convert document batch...
2025-11-17 01:09:24,303 - INFO - Initializing pipeline for SimplePipeline with options hash 995a146ad601044538e6a923bea22f4e
2025-11-17 01:09:24,359 - INFO - Loading plugin 'docling_defaults'
2025-11-17 01:09:24,365 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-11-17 01:09:24,367 - INFO - Processing document New Text Document.csv
2025-11-17 01:09:24,369 - INFO - Parsing CSV with delimiter: ","
2025-11-17 01:09:24,370 - INFO - Detected 1 lines
2025-11-17 01:09:24,375 - INFO - Finished converting document New Text Document.csv in 0.09 sec.
2025-11-17 01:09:26,061 - INFO - HTTP Request: POST https://router.huggingface.co/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-17 01:09:51,677 - INFO - detected formats: [<InputFormat.CSV: 